In [17]:
import geopandas as gpd
import pyproj
import os
import zipfile
import numpy as np
import math
import rasterio
from shapely.geometry import Polygon,shape, mapping
from shapely.affinity import rotate
import fiona

In [18]:
def find_shapefile_path(base_path):
    """
    Finds the path to a shapefile within a single unknown subfolder
    under the given base path.

    Args:
        base_path (str): The base path to the folder.

    Returns:
        str: The full path to the shapefile, or None if no shapefile is found.
    """
    full_path = os.path.join(base_path, "data", "plotlocation_shapefile")
    subfolders = [f.path for f in os.scandir(full_path) if f.is_dir()]
    print(subfolders)

    if len(subfolders) != 1:
        return None  # No subfolder or multiple subfolders found

    shapefile_path = None
    for file in os.listdir(subfolders[0]):  # Corrected here
        if file.endswith(".shp"):
            shapefile_path = os.path.join(subfolders[0], file)
            break

    return shapefile_path

In [19]:
def find_image_file_path(base_path):
    """
    Finds a file without an extension directly inside the "hs_raw_image" folder.

    Args:
        base_path (str): The base path to the folder.

    Returns:
        str: The full path to the file, or None if no matching file is found.
    """
    full_path = os.path.join(base_path, "data", "hs_raw_image")

    # Ensure the directory exists
    if not os.path.exists(full_path):
        print(f"Error: Directory does not exist - {full_path}")
        return None

    print(f"Searching in: {full_path}")

    file_path = None
    for file in os.listdir(full_path):  # Directly list files in full_path
        if "." not in file:  # Check if the file has no extension
            file_path = os.path.join(full_path, file)
            break

    if file_path is None:
        print("Error: No file without an extension found.")

    return file_path

In [20]:
def get_rotation_angle(geotransform):
    """
    Calculate the rotation angle from the geotransform.
    """
    # Extract the relevant geotransform values
    rotation_x = geotransform[2]
    pixel_width = geotransform[1]
    rotation_y = geotransform[4]
    pixel_height = geotransform[5]

    # Calculate the rotation angle using atan2
    rotation_angle = math.degrees(math.atan2(rotation_y, pixel_width))

    return rotation_angle

In [21]:
def rotate_shapefile(input_shapefile, output_shapefile, angle):
    """
    Rotate the shapefile by a given angle.
    """
    with fiona.open(input_shapefile, 'r') as src:
        # Define a new schema
        schema = src.schema.copy()
        
        with fiona.open(output_shapefile, 'w', driver=src.driver, crs=src.crs, schema=schema) as dst:
            for feature in src:
                geom = shape(feature['geometry'])
                centroid = geom.centroid
                rotated_geom = rotate(geom, angle, origin=(centroid.x, centroid.y))
                
                new_feature = {
                    'type': 'Feature',
                    'geometry': mapping(rotated_geom),
                    'properties': feature['properties']
                }
                dst.write(new_feature)
                
    print(f"Rotated shapefile saved to {output_shapefile}")

In [22]:
def create_bounding_square_shapefile(input_shapefile, input_hyperspectral_image, square_sidelength=None, buffer_m=10, manual_center=None):
    """
    Create a square shapefile that fully covers all points in an input shapefile,
    with an additional buffer.

    Parameters:
    - input_shapefile: Path to the input point shapefile.
    - input_hyperspectral_image: Path to the hyperspectral image (for rotation reference).
    - square_sidelength: Optional side length of the square in meters.
    - buffer_m: Buffer in meters (default is 10m).
    - manual_center: Optional (lon, lat) tuple for setting a custom square center.

    Returns:
    - None
    """
    # Load the point shapefile
    gdf = gpd.read_file(input_shapefile)
    gdf = gdf[gdf.geometry.notnull()]
    
    # Get the CRS from the input shapefile
    input_crs = gdf.crs
    if input_crs is None:
        raise ValueError("The input shapefile has no defined CRS.")
    
    # Ensure it contains only point geometries
    #if not all(gdf.geometry.type == "Point"):
    #    raise ValueError("The input shapefile must contain only Point geometries.")

    # Use manual center if provided, otherwise compute centroid
    if manual_center:
        center_x, center_y = manual_center
    else:
        centroid = gdf.geometry.union_all().centroid  # Updated for GeoPandas 0.14+
        center_x, center_y = centroid.x, centroid.y
    
    # Determine the appropriate UTM zone
    if input_crs.is_geographic:
        utm_zone = math.floor((center_x + 180) / 6) + 1  # Fixed for negative longitudes
        utm_crs = pyproj.CRS(f"EPSG:{32600 + utm_zone if center_y >= 0 else 32700 + utm_zone}")
    else:
        utm_crs = input_crs
    
    project_to_utm = pyproj.Transformer.from_crs(input_crs, utm_crs, always_xy=True).transform
    project_to_input = pyproj.Transformer.from_crs(utm_crs, input_crs, always_xy=True).transform
    
    center_utm = project_to_utm(center_x, center_y)
    
    # Transform all points to UTM if not using manual center
    if not manual_center:
        gdf_utm = gdf.to_crs(utm_crs)
        centroid_utm = gdf_utm.geometry.union_all().centroid
        distances = gdf_utm.geometry.distance(centroid_utm)
        if distances.isna().any():
            raise ValueError("Distance calculation resulted in NaN values. Check input data.")
        max_distance = max(distances)

    if square_sidelength is None:
        side_length = 2 * (max_distance + buffer_m)
    else:
        conversion_factor = 0.3375  # This should be validated
        side_length = square_sidelength / conversion_factor
            #side_length = square_sidelength
    #else:
    #    side_length = square_sidelength if square_sidelength else 100  # Default size if not given
    
    print(f'side_length: {side_length}')

    # Compute the four corners of the square in UTM coordinates
    half_side = side_length / 2
    square_utm_coords = [
        (center_utm[0] - half_side, center_utm[1] - half_side),
        (center_utm[0] + half_side, center_utm[1] - half_side),
        (center_utm[0] + half_side, center_utm[1] + half_side),
        (center_utm[0] - half_side, center_utm[1] + half_side),
        (center_utm[0] - half_side, center_utm[1] - half_side)  # Ensure polygon closure
    ]
    
    # Validate UTM coordinates
    if any(np.isnan(coord[0]) or np.isnan(coord[1]) for coord in square_utm_coords):
        raise ValueError("Generated square contains NaN values. Check UTM transformations.")
    
    square_utm = Polygon(square_utm_coords)
    
    # Convert the square back to the original CRS
    square_input_coords = [project_to_input(x, y) for x, y in square_utm.exterior.coords]
    square_input = Polygon(square_input_coords)
    
    # Validate final polygon
    if not square_input.is_valid:
        raise ValueError("Generated square is invalid after re-projection. Check calculations.")
    
    # Create GeoDataFrame
    gdf_square = gpd.GeoDataFrame(index=[0], crs=input_crs, geometry=[square_input])
    
    # Extract the directory containing the file
    input_filename = os.path.splitext(os.path.basename(input_shapefile))[0]
    output_folder_path = os.path.dirname(input_shapefile)

    # Create output directory
    os.makedirs(output_folder_path, exist_ok=True)
    
    # Define output file paths
    shapefile_path = os.path.join(output_folder_path, f"{input_filename}_cutshape.shp")
    output_shapefile_rotated = os.path.join(output_folder_path, f"{input_filename}_cutshape_rotated.shp")

    # Save to shapefile
    gdf_square.to_file(shapefile_path)
    print(f"Bounding square shapefile saved to {shapefile_path}")

    # Read geotransformation parameters from the hyperspectral image
    with rasterio.open(input_hyperspectral_image) as dataset:
        geotransform = dataset.transform

    # Calculate rotation angle from geotransform
    rotation_angle = get_rotation_angle(geotransform)
    print(f"Rotation angle derived from geotransformation: {rotation_angle} degrees")

    # Rotate shapefile by the derived angle
    rotate_shapefile(shapefile_path, output_shapefile_rotated, rotation_angle)

In [25]:
input_path = "D:/MasterThesis/final_hs_data_folder/AN_TJ_Plot2_ang20220711t003358rfl"  # Replace with your actual shapefile

input_shapefile = find_shapefile_path(input_path)

input_hs_image = find_image_file_path(input_path)

# Check if files exist
if os.path.exists(input_shapefile):
    print(f"Shapefile exists: {input_shapefile}")
else:
    print(f"Shapefile not found: {input_shapefile}")

# Check if files exist
if os.path.exists(input_hs_image):
    print(f"Image exists: {input_hs_image}")
else:
    print(f"Image not found: {input_hs_image}")

['D:/MasterThesis/final_hs_data_folder/AN_TJ_Plot1_ang20220711t002111rfl\\data\\plotlocation_shapefile\\AN_TJ_Plot1']
Searching in: D:/MasterThesis/final_hs_data_folder/AN_TJ_Plot1_ang20220711t002111rfl\data\hs_raw_image
Shapefile exists: D:/MasterThesis/final_hs_data_folder/AN_TJ_Plot1_ang20220711t002111rfl\data\plotlocation_shapefile\AN_TJ_Plot1\AN_TJ_Plot1.shp
Image exists: D:/MasterThesis/final_hs_data_folder/AN_TJ_Plot1_ang20220711t002111rfl\data\hs_raw_image\ang20220711t002111_rfl_v2aa2_img


In [26]:
'''
create_bounding_square_shapefile(
    input_shapefile=input_shapefile,
    input_hyperspectral_image=input_hs_image,
    square_sidelength=3000,
    manual_ce
    nter=(-18066876.035,10436180.546)  # Alaska coordinates
)
'''
create_bounding_square_shapefile(
    input_shapefile=input_shapefile,
    input_hyperspectral_image=input_hs_image,
    buffer_m=20
)

side_length: 2771.0966505497518
Bounding square shapefile saved to D:/MasterThesis/final_hs_data_folder/AN_TJ_Plot1_ang20220711t002111rfl\data\plotlocation_shapefile\AN_TJ_Plot1\AN_TJ_Plot1_cutshape.shp
Rotation angle derived from geotransformation: -58.00000000000001 degrees
Rotated shapefile saved to D:/MasterThesis/final_hs_data_folder/AN_TJ_Plot1_ang20220711t002111rfl\data\plotlocation_shapefile\AN_TJ_Plot1\AN_TJ_Plot1_cutshape_rotated.shp


def create_bounding_square_shapefile(input_shapefile,input_hyperspectral_image, square_sidelength=None, buffer_m=10):
    """
    Create a square shapefile that fully covers all points in an input shapefile,
    with an additional buffer.

    Parameters:
    - input_shapefile: Path to the input point shapefile.
    - square_sidelength: length of the side of the square (optional)
    - buffer_m: Buffer in meters (default is 10m).

    Returns:
    - None
    """
    # Load the point shapefile
    gdf = gpd.read_file(input_shapefile)
    
    # Get the CRS from the input shapefile
    input_crs = gdf.crs
    if input_crs is None:
        raise ValueError("The input shapefile has no defined CRS.")
    
    # Ensure it contains only point geometries
    if not all(gdf.geometry.type == "Point"):
        raise ValueError("The input shapefile must contain only Point geometries.")
    
    # Compute centroid of all points
    centroid = gdf.geometry.union_all().centroid  # Updated for GeoPandas 0.14+
    center_x, center_y = centroid.x, centroid.y
    
    # Transform all points to projected CRS (if not already projected)
    if input_crs.is_geographic:
        utm_zone = (math.floor((center_x + 180) / 6) % 60) + 1
        if center_y >= 0:
            utm_crs = pyproj.CRS(f'EPSG:326{utm_zone:02d}')  # Northern hemisphere
        else:
            utm_crs = pyproj.CRS(f'EPSG:327{utm_zone:02d}')  # Southern hemisphere
    else:
        utm_crs = input_crs
    
    project_to_utm = pyproj.Transformer.from_crs(input_crs, utm_crs, always_xy=True).transform
    project_to_input = pyproj.Transformer.from_crs(utm_crs, input_crs, always_xy=True).transform
    
    center_utm = project_to_utm(center_x, center_y)
    
    # Transform all points to UTM
    gdf_utm = gdf.to_crs(utm_crs)
    
    # Compute max distance from centroid to any point
    #centroid_utm = gdf_utm.geometry.unary_union.centroid
    centroid_utm = gdf_utm.geometry.union_all().centroid

    distances = gdf_utm.geometry.distance(centroid_utm)
    if distances.isna().any():
        raise ValueError("Distance calculation resulted in NaN values. Check input data.")
    max_distance = max(distances)

    if square_sidelength is None:
        # Compute square side length
        side_length = 2 * (max_distance + buffer_m)  # Ensure square covers all points + buffer
    else:
        conversion_factor = 0.3375
        side_length = square_sidelength / conversion_factor
   
    # Compute the four corners of the square in UTM coordinates
    half_side = side_length / 2
    square_utm_coords = [
        (center_utm[0] - half_side, center_utm[1] - half_side),
        (center_utm[0] + half_side, center_utm[1] - half_side),
        (center_utm[0] + half_side, center_utm[1] + half_side),
        (center_utm[0] - half_side, center_utm[1] + half_side),
        (center_utm[0] - half_side, center_utm[1] - half_side)  # Ensure polygon closure
    ]
    
    # Validate UTM coordinates
    if any(np.isnan(coord[0]) or np.isnan(coord[1]) for coord in square_utm_coords):
        raise ValueError("Generated square contains NaN values. Check UTM transformations.")
    
    square_utm = Polygon(square_utm_coords)
    
    # Convert the square back to the original CRS
    square_input_coords = [project_to_input(x, y) for x, y in square_utm.exterior.coords]
    square_input = Polygon(square_input_coords)
    
    # Validate final polygon
    if not square_input.is_valid:
        raise ValueError("Generated square is invalid after re-projection. Check calculations.")
    
    # Create GeoDataFrame
    gdf_square = gpd.GeoDataFrame(index=[0], crs=input_crs, geometry=[square_input])
    
    
    # Extract the directory containing the file
    input_filename = os.path.splitext(os.path.basename(input_shapefile))[0]
    #print(f'input_filename: {input_filename}')

    #output_folder_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(input_shapefile))),"cut_shapefile", f"{input_filename}")
    output_folder_path = os.path.dirname(input_shapefile)
    #print(f'output_folder_path: {output_folder_path}')

    # Create output directory
    os.makedirs(output_folder_path, exist_ok=True)
    
    # Define output file paths
    shapefile_path = os.path.join(output_folder_path, f"{input_filename}_cutshape.shp")
    zipfile_path = os.path.join(output_folder_path, f"{input_filename}_cutshape.zip")
    output_shapefile_rotated = os.path.join(output_folder_path, f"{input_filename}_cutshape_rotated.shp")

    # Save to shapefile
    gdf_square.to_file(shapefile_path)
    print(f"Bounding square shapefile saved to {shapefile_path}")

    # Read geotransformation parameters from the hyperspectral image
    with rasterio.open(input_hyperspectral_image) as dataset:
        geotransform = dataset.transform

    # Calculate rotation angle from geotransform
    rotation_angle = get_rotation_angle(geotransform)
    print(f"Rotation angle derived from geotransformation: {rotation_angle} degrees")

    # Rotate shapefile by the derived angle
    rotate_shapefile(shapefile_path, output_shapefile_rotated, rotation_angle)

    '''
    # Create a zip archive of the folder
    with zipfile.ZipFile(zipfile_path, 'w') as zipf:
        for root, _, files in os.walk(output_folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, output_folder_path)
                zipf.write(file_path, arcname=arcname)
    
    print(f"Zipfile created at {zipfile_path}")
    '''

input_path = "D:/MasterThesis/final_hs_data_folder/FRST_AK_Plot5_ang20220710t002801rfl"  # Replace with your actual shapefile

input_shapefile = find_shapefile_path(input_path)

input_hs_image = find_image_file_path(input_path)

# Check if files exist
if os.path.exists(input_shapefile):
    print(f"Shapefile exists: {input_shapefile}")
else:
    print(f"Shapefile not found: {input_shapefile}")

# Check if files exist
if os.path.exists(input_hs_image):
    print(f"Image exists: {input_hs_image}")
else:
    print(f"Image not found: {input_hs_image}")

create_bounding_square_shapefile(input_shapefile,input_hs_image, 3000, buffer_m=10)